# GCOM Practice

This notebook demonstrates connecting, downloading and translating GCOM satellite data from [G-Portal](https://gportal.jaxa.jp/gpr/).

## Prerequisites

You're required to have G-Portal account. You can register from [here](https://gportal.jaxa.jp/gpr/user/regist1).

# Connect to FTP server

The first step is to connect to the FTP server where they host the satellite data in HDF5 format.
Here I use `ftplib` module.

In [1]:
# See: https://gportal.jaxa.jp/gpr/assets/mng_upload/COMMON/upload/GPortalUserManual_ja.pdf for the details.
host = 'ftp.gportal.jaxa.jp'
user = ''
passwd = 'anonymous' # fixed password for all the users.

In [ ]:
from ftplib import FTP
ftp = FTP(host)
ftp.login(user, passwd)

Let's check if it's connected by listing directory contents.

In [12]:
ftp.dir()

drwxr-xr-x   5 3498     3498         4096 May 29  2018 nrt
drwxr-xr-x 317 3498     3498       131072 Oct 30 08:37 order
drwxr-xr-x 1647 3498     3498       131072 Oct 30 12:54 process
drwxr-xr-x  17 3498     3498         4096 Mar 18  2020 standard


## Download GCOM data

Now let's try downloading GCOM-C LST (Land Surface Temperature) L3 data specifying year/month/date.

In [50]:
def download_gcomc_lst_product(ftp: FTP, year: int, month: int, date: int):
    """
    Downloads GCOM-C LST (Land Surface Temperature) product ver. 3
    Parameters:
    - ftp
    - year
    - month
    - date
    """
    base = 'downloads'

    import os
    dir = "standard/GCOM-C/GCOM-C.SGLI/L3.LAND.LST_/3/{:d}/{:02d}".format(year, month)
    os.makedirs(os.path.join(base, dir), exist_ok=True)

    # Ascending data
    filename_a = 'GC1SG1_{:d}{:02d}{:02d}A01D_D0000_3MSG_LST_F_3000.h5'.format(year, month, date)
    with open(os.path.join(base, dir, filename_a), 'wb') as f:
        ftp.retrbinary(f"RETR /{dir}/{filename_a}", f.write)

    # Descending data
    filename_d = 'GC1SG1_{:d}{:02d}{:02d}D01D_D0000_3MSG_LST_F_3000.h5'.format(year, month, date)
    with open(os.path.join(base, dir, filename_d), 'wb') as f:
        ftp.retrbinary(f"RETR /{dir}/{filename_d}", f.write)

In [53]:
for date in range(1, 5):
    # try downloading for the whole September data
    download_gcomc_lst_product(ftp, 2023, 9, date)

## GDAL Translate

Now translate the downloaded HDF5 file into GeoTIFF using GDAL translate.

In [49]:
def translate_from_hdf_to_gtiff(year: int, month: int, date: int):
    """
    Translates downloaded HDF file into GeoTIFF file
    Parameters:
    - year
    - month
    - date
    """
    import os
    from osgeo import gdal
    downloaded_dir = "downloads/standard/GCOM-C/GCOM-C.SGLI/L3.LAND.LST_/3/{:d}/{:02d}".format(year, month)
    output_dir = "output/standard/GCOM-C/GCOM-C.SGLI/L3.LAND.LST_/3/{:d}/{:02d}".format(year, month)
    os.makedirs(output_dir, exist_ok=True)

    # Ascending data
    downloaded_filename_a = "GC1SG1_{:d}{:02d}{:02d}A01D_D0000_3MSG_LST_F_3000.h5".format(year, month, date)
    downloaded_a = gdal.Open(os.path.join(downloaded_dir, downloaded_filename_a), gdal.GA_ReadOnly)
    image_ds_a = gdal.Open(downloaded_a.GetSubDatasets()[0][0], gdal.GA_ReadOnly)
    image_ds_a.GetRasterBand(1).SetScale(0.02)
    image_ds_a.GetRasterBand(1).SetOffset(0)
    
    output_filename_a = "GC1SG1_{:d}{:02d}{:02d}A01D_D0000_3MSG_LST_F_3000.tif".format(year, month, date)
    gdal.Translate(
        os.path.join(output_dir, output_filename_a),
        image_ds_a,
        format="GTiff",
        outputSRS="EPSG:4326",
        outputBounds=[-180, 90, 180, -90],
        noData=65535,
        creationOptions=['COMPRESS=LZW'])

    # Descending data
    downloaded_filename_d = "GC1SG1_{:d}{:02d}{:02d}D01D_D0000_3MSG_LST_F_3000.h5".format(year, month, date)
    downloaded_d = gdal.Open(os.path.join(downloaded_dir, downloaded_filename_d), gdal.GA_ReadOnly)
    image_ds_d = gdal.Open(downloaded_d.GetSubDatasets()[0][0], gdal.GA_ReadOnly)
    image_ds_d.GetRasterBand(1).SetScale(0.02)
    image_ds_d.GetRasterBand(1).SetOffset(0)
    
    output_filename_d = "GC1SG1_{:d}{:02d}{:02d}D01D_D0000_3MSG_LST_F_3000.tif".format(year, month, date)
    gdal.Translate(
        os.path.join(output_dir, output_filename_d),
        image_ds_d,
        format="GTiff",
        outputSRS="EPSG:4326",
        outputBounds=[-180, 90, 180, -90],
        noData=65535,
        creationOptions=['COMPRESS=LZW'])

In [54]:
for date in range(1, 5):
    # translate HDF into GeoTIFF
    translate_from_hdf_to_gtiff(2023, 9, date)